In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [36]:
class Bottleneck(nn.Module):

    def __init__(self, in_channels, reduced_channels, out_channels, stride=1):
        super(Bottleneck, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, reduced_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(reduced_channels, reduced_channels, kernel_size=3, bias=False),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(reduced_channels, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels),
        )
        self.shortcut = nn.Sequential(
            nn.Conv2d(in_channels,out_channels,kernel_size=1, stride=stride, bias=False)
        )
        self.final_relu = nn.ReLU(inplace=True)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out) + self.shortcut(x)
        out = self.final_relu(out)
        return out


In [37]:
class ResNet50(nn.Module):

    def __init__(self, num_classes=1000):
        super(ResNet50, self).__init__()
        self.num_blocks = [3,4,6,3]
        # Conv1 layer: 224*224*3 -> 112*112*64
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=4),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.conv2 = self._make_layer(in_channels=64, reduced_channels=64, out_channels=256, num_blocks=self.num_blocks[0], stride=1)
        self.conv3 = self._make_layer(in_channels=256, reduced_channels=128, out_channels=512, num_blocks=self.num_blocks[1], stride=2)
        self.conv4 = self._make_layer(in_channels=512, reduced_channels=256, out_channels=1024, num_blocks=self.num_blocks[2], stride=2)
        self.conv5 = self._make_layer(in_channels=1024, reduced_channels=512, out_channels=2048, num_blocks=self.num_blocks[3], stride=2)
        
        # FC layer, after applied 'avg pooling'
        self.fc1 = nn.Sequential(
            nn.Linear(in_features=2048, out_features=1000)
        )

    def _make_layer(self, in_channels, reduced_channels, out_channels, num_blocks, stride=1):
        layers = []
        strides = [stride] + [1]*(num_blocks-1)

        for s in strides:
            bottleneck_layer = Bottleneck(in_channels, reduced_channels, out_channels, s)
            layers.append(bottleneck_layer)
            in_channels = out_channels *4
        
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.conv5(out)
        out = out.view(out.size(0), -1)
        out = nn.AvgPool2d(out, kernel_size=4)
        return out

In [38]:
from torchsummary import summary

resnet = ResNet50().cuda()
summary(resnet, input_size=(3, 224, 224))

RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 11.78 GiB total capacity; 7.46 GiB already allocated; 5.94 MiB free; 7.48 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF